In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# unzip
import zipfile, os, shutil

dataset = '/content/gdrive/My Drive/Colab_Notebooks/DL/datasets/cifar10.zip'
dst_path = '/content/cifa10'
dst_file = os.path.join(dst_path, 'cifar10.zip')

if not os.path.exists(dst_path):
  os.makedirs(dst_path)

# copy zip file
shutil.copy(dataset, dst_file)
  
with zipfile.ZipFile(dst_file, 'r') as file:
  file.extractall(dst_path)

Mounted at /content/gdrive


In [3]:
train_dir = os.path.join(dst_path, 'cifar10/train')

test_dir = os.path.join(dst_path, 'cifar10/test')

print('total training  images:', len(os.listdir(train_dir)))

print('total test images:', len(os.listdir(test_dir)))


total training  images: 10
total test images: 10


In [4]:
# path
train_path = "/content/cifa10/cifar10/train/"
test_path = "/content/cifa10/cifar10/test/"

In [5]:
# library
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import scipy



# load Neural Network Model Library => condition 3 of assignment
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
# Set generator with rescaler(1./255) -> condition 2 of assignment
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

# make a loading data flow from path. It generates data at each batch sizes -> condition 1 of assignment
batch_size_num = 100
train_generator = train_datagen.flow_from_directory(train_path,
                                                    classes=sorted(os.listdir(train_path)),
                                                    batch_size = batch_size_num,
                                                    target_size = (256, 256),
                                                    subset="training",
                                                    class_mode='categorical')

valid_generator = train_datagen.flow_from_directory(train_path, 
                                                    classes=sorted(os.listdir(train_path)), 
                                                    batch_size = batch_size_num, 
                                                    target_size=(256, 256), 
                                                    subset="validation",
                                                    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(test_path, 
                                                  classes=sorted(os.listdir(test_path)),
                                                  batch_size = 100,
                                                  target_size = (256, 256),
                                                  class_mode='categorical')

print()
# check shape of data shape
print("check shape of data shape")
for x_data, class_data in train_generator:
    print(f"input data shape from train_generator: {x_data.shape}")
    print(f"class data shape from train_generator: {class_data.shape}")
    break

for x_data, class_data in valid_generator:
    print(f"input data shape from valid_generator: {x_data.shape}")
    print(f"class data shape from valid_generator: {class_data.shape}")
    break
    
for x_data, class_data in test_generator:
    print(f"input data shape from test_generator: {x_data.shape}")
    print(f"class data shape from test_generator: {class_data.shape}")
    break    

Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.

check shape of data shape
input data shape from train_generator: (100, 256, 256, 3)
class data shape from train_generator: (100, 10)
input data shape from valid_generator: (100, 256, 256, 3)
class data shape from valid_generator: (100, 10)
input data shape from test_generator: (100, 256, 256, 3)
class data shape from test_generator: (100, 10)


In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# MAML 모델 구성
model = keras.Sequential([
    layers.Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=(256, 256, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'),
    layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'),
    layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    # layers.Dense(10, activation='softmax')
    layers.Dense(5, activation='softmax')  # 클래스 개수에 맞게 변경
])

In [44]:
# MAML 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
# MAML 알고리즘 설정
meta_learning_rate = 0.001
inner_learning_rate = 0.01
num_tasks = 450
num_inner_updates = 1

In [48]:
# 메타-학습 반복
for meta_iteration in range(num_tasks):
    # task_indices = np.random.choice(len(x_train), num_tasks, replace=False)

    # 작업 별로 초기 파라미터 설정
    model_clone = keras.models.clone_model(model)
    model_clone.set_weights(model.get_weights())
    model_clone.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    # train_generator의 클래스 개수
    num_classes = len(train_generator.class_indices)

    for inner_iteration in range(num_inner_updates):
       # 랜덤하게 5개의 클래스 선택
      random_classes = np.random.choice(num_classes, size=5, replace=False)
    
      # 선택한 클래스에 해당하는 데이터로 작업 데이터 생성
      x_task_train = []
      y_task_train = []
    for cls in random_classes:
        # 해당 클래스에 속하는 이미지들을 가져옴
        images, labels = train_generator.next()
        # 클래스에 해당하는 이미지와 레이블을 작업 데이터에 추가
        x_task_train.append(images)
        y_task_train.append(labels)

        # 작업 데이터를 numpy 배열로 변환
    x_task_train = np.concatenate(x_task_train, axis=0)
    y_task_train = np.concatenate(y_task_train, axis=0)

    selected_classes = np.random.choice(10, size=5, replace=False)

    # 선택된 클래스들에 해당하는 인덱스를 추출하여 새로운 x_task_train과 y_task_train 생성
    selected_indices = np.isin(y_task_train.argmax(axis=1), selected_classes)

    x_task_train_selected = x_task_train[selected_indices]
    y_task_train_selected = y_task_train[selected_indices][:, selected_classes]

    for inner_iteration in range(num_inner_updates):
        # x_task_train, y_task_train = train_generator.next()

        # 그래디언트 계산을 위해 작업 데이터로 모델 학습
        model_clone.train_on_batch(x_task_train_selected, y_task_train_selected)
        
        # 미세 조정을 위해 학습한 모델 파라미터를 업데이트
        model_weights = model_clone.get_weights()
        model.set_weights(model_weights)

    # 메타-학습을 위해 초기 파라미터를 업데이트
    model_weights = model.get_weights()
    for i, weight in enumerate(model_weights):
        model_weights[i] = weight + meta_learning_rate * (weight - model_clone.get_weights()[i])
    model.set_weights(model_weights)


    # 선택한 클래스에 해당하는 데이터로 작업 데이터 생성
    x_task_val = []
    y_task_val = []
    # 현재 meta_iteration에서의 val 데이터셋에 대한 손실과 정확도 출력
    for cls in random_classes:
        # 해당 클래스에 속하는 이미지들을 가져옴
        images1, labels1 = valid_generator.next()
        # 클래스에 해당하는 이미지와 레이블을 작업 데이터에 추가
        x_task_val.append(images)
        y_task_val.append(labels)

        # 작업 데이터를 numpy 배열로 변환
    x_task_val = np.concatenate(x_task_val, axis=0)
    y_task_val = np.concatenate(y_task_val, axis=0)

    x_task_val_selected = x_task_val[selected_indices]
    y_task_val_selected = y_task_val[selected_indices][:, selected_classes]

    # x_val, y_val = valid_generator.next()
    val_loss, val_acc = model.evaluate(x_task_val_selected, y_task_val_selected)
    print(f"Meta Iteration: {meta_iteration + 1}  val Loss: {val_loss:.4f} val Acc: {val_acc:.4f}")

ValueError: ignored

In [ ]:
x_test, y_test = test_generator.next()

In [ ]:
# MAML 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test Accuracy:", test_acc)

In [ ]:
model.predict(x_test)

In [ ]:
model.save("aaaa.h5")